In [ ]:
import jax

dev = jax.devices()
key = jax.random.PRNGKey(0)

print(dev)
print(key)


[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
[0 0]


In [ ]:
#!nvidia-smi
#!pip install git+https://github.com/sanchit-gandhi/whisper-jax.git

In [ ]:
from jax import numpy as jnp
from whisper_jax import FlaxWhisperPipline

pipeline = FlaxWhisperPipline("openai/whisper-large-v2", dtype=jnp.float16)

In [ ]:
text = pipeline("harvard.wav")
text

{'text': ' The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.'}

In [ ]:
html = """
<html>
<head>
<style>
    button { font-size: 18px; font-family: monospace; }
    audio { display: block; margin: 10px 0; }
    .record-button { background-color: red; color: white; border-radius: 25px; border: solid 1px darkred; padding: 10px 25px; cursor: pointer; transition: 50ms; }
    .record-button:hover { background-color: #d30000; }
    .record-button.on { background-color: #00d300; border: solid 1px darkgreen; }
    .record-button.on:hover { background-color: #00a200; }
</style>
</head>
<body>
<div>
    <button id="record" class="record-button">Record</button>
</div>
<script>
    let recording = false;
    let recorder = null;
    let data = null;

    const button = document.getElementById("record");

    function record(stream, resolve, reject) {
        button.classList.add("on");
        button.innerText = "On Air";
        recording = true;

        recorder = new MediaRecorder(stream);
        recorder.ondataavailable = function(e) {
            const url = URL.createObjectURL(e.data);
            const preview = document.createElement('audio');
            preview.controls = true;
            preview.src = url;
            document.body.appendChild(preview);

            const reader = new FileReader();
            reader.readAsDataURL(e.data); 
            reader.onloadend = function() { resolve(reader.result); };
        };
        recorder.start();
    }

    function toggle(resolve, reject) {
        if (recording) {
            button.classList.remove("on");
            button.innerText = "Record";
            button.style.display = "none";
            recording = false;
            recorder.stop();
        } else {
            navigator.mediaDevices.getUserMedia({audio: true})
                .then(stream => record(stream, resolve, reject))
                .catch(console.error);
        }
    }

    const promised = new Promise(function(resolve, reject) {
        button.addEventListener("click", function(e) { toggle(resolve, reject); });
    });
</script>
</body>
</html>
"""

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode

display(HTML(html))
data = eval_js("promised")

In [ ]:
binary = b64decode(data.split(',')[1])

with open("recording.ogg", "wb+") as f:
    f.write(binary)

transcription = pipeline("recording.ogg")
transcription

{'text': ' Что тебе сказать?'}